In [1]:
import pandas as pd

In [4]:
import numpy as np

In [12]:
df_ratings = pd.read_table('ratings_ml.dat', sep='::', names=['UserID','MovieID','Rating','Timestamp'])
df_movies = pd.read_table('movies_ml.dat', sep='::', names=['MovieID','Title','Genres'])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [13]:
df_ratings.iloc[0][2]

5

In [14]:
df_ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [15]:
df_movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
df_movies_join_ratings = pd.merge(df_ratings, df_movies, on='MovieID')

In [17]:
df_movies_join_ratings.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [19]:
del df_movies_join_ratings['Timestamp']
del df_movies_join_ratings['Genres']

In [20]:
df_movies_join_ratings.head()

,UserID,MovieID,Rating,Title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975)
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975)
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975)
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975)


In [22]:
df_movies_join_ratings['Rating'].value_counts()

4    348971
3    261197
5    226310
2    107557
1     56174
Name: Rating, dtype: int64

In [37]:
from surprise import Dataset, Reader, KNNBasic, SVD, KNNBaseline
from surprise.model_selection import train_test_split
from surprise import accuracy

In [26]:
df = df_ratings[['UserID', 'MovieID', 'Rating']]

In [27]:
df.columns = ['uid', 'iid', 'rating']

In [28]:
df.head()

,uid,iid,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [29]:
reader = Reader(rating_scale=(1, 5))

In [30]:
dataset = Dataset.load_from_df(df, reader)

In [31]:
trainset, testset = train_test_split(dataset, test_size=0.2)

User-to-Item

In [32]:
algo = KNNBasic(k=40, sim_options={'name': 'cosine', 'user_based': True})

In [33]:
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [34]:
predictions = algo.test(testset)

In [35]:
accuracy.rmse(predictions)

RMSE: 0.9754


0.9754075135275965

In [38]:
algo2 = KNNBaseline(k=40, sim_options={'name': 'cosine', 'user_based': True})

In [39]:
algo2.fit(trainset)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [40]:
predictions2 = algo2.test(testset)

In [41]:
accuracy.rmse(predictions2)

RMSE: 0.8994


0.8993657574361804

In [42]:
algo3 = KNNBaseline(k=40, sim_options={'name': 'pearson_baseline', 'user_based': True})

In [43]:
algo3.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [44]:
predictions3 = algo3.test(testset)

In [45]:
accuracy.rmse(predictions3)

RMSE: 0.8678


0.8677603304594911

для юзера 1

In [88]:
algo3.trainset.to_inner_uid(1)

2682

In [89]:
target_movies = algo3.get_neighbors(iid=algo3.trainset.to_inner_uid(1), k=5)
target_movies

[1803, 1408, 272, 1485, 2007]

In [90]:
algo3.trainset.to_raw_iid(1803)

2372

In [91]:
algo3.trainset.to_inner_iid(2372)

1803

In [92]:
for item in target_movies:
    res = df_movies[df_movies['MovieID'] == algo3.trainset.to_raw_iid(item)]
    print(item, res['MovieID'].tolist()[0], res['Title'].tolist()[0])

1803 2372 Fletch Lives (1989)
1408 678 Some Folks Call It a Sling Blade (1993)
272 181 Mighty Morphin Power Rangers: The Movie (1995)
1485 930 Notorious (1946)
2007 2205 Mr. & Mrs. Smith (1941)


In [99]:
algo3.predict(algo.trainset.to_inner_uid(1), algo.trainset.to_inner_iid(2372))

Prediction(uid=2682, iid=1803, r_ui=None, est=3.119279965611367, details={'was_impossible': False})